In [4]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

# engine powering Astra DB integreation in Langchain
# DB connection initilization
import cassio

/Users/syshin/Desktop/Syshin/Study/MLDL/LangChain/crash_course/pdf_query/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install PyPDF2

In [6]:
from PyPDF2 import PdfReader

In [7]:
!pip install python-dotenv

In [8]:
from dotenv import load_dotenv
import os

load_dotenv() 

ASTRA_DB_ID = os.environ['ASTRA_DB_ID']
ASTRA_DB_APPLICATION_TOKEN = os.environ['ASTRA_DB_APPLICATION_TOKEN']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


In [9]:
!pip install --upgrade astrapy

In [10]:
pdfreader = PdfReader('sample.pdf')

In [11]:
from typing_extensions import Concatenate

# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

raw_text

'2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 1/5\n사람인 인성검사 결과응시자\n신승엽희망직무\nIT·인터넷검사일자\n2024.01.21\n표준점수 인성검사 응시자 집단에서 나의 상대적인 위치를 나타내기 위해 산출하는 점수\n백분위 나보다 낮은 점수를 받은 사람을 백분율로 표기\n예) 점수가 95%라면 내 점수보다 낮은 사람이 95% 있다는 것을 의미\n응답신뢰도 일관적인 답변을 하지 않았거나 실제 자신의 성향보다 더 긍정적인 방향으로 응답하면 신뢰도가 낮아질 수 있음직무적합(IT·인터넷) : 높음\nIT·인터넷직에서는 작은 것도 놓치지 않는 세밀함, 논리적이고 체계적인 사고력 및 업무적으로 타인에게 신뢰감을 심어줄 수\n있는 자세, 주도적인 태도로 새롭거나 대안적인 방안을 제시하려는 자세가 요구됩니다.\n신승엽님은 해당 직무에 대한 적합도가 매우 높아 주어진 직무 상황에 잘 적응하며, 우수한 역량을 발휘할 가능성이 높습니다.* 본 검사는 응답을 토대로 작성되었으며, 본인의 직무 적합도를 확인할 수 있습니다. 종합 결과\n표준점수\n69 점백분위\n93 %응답신뢰도\n낮음\n인성 요인 비교\n정직겸손성\n정서안정성\n외향성\n원만성성실성개방성내 점수 평균 점수\n요인별 평가2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 2/5정직겸손성 자신에게 유리한 방향으로 의사 결정하거나 사람들을 조정하지 않고, 물질적인 것에 과하게 욕심을 내지 않으며 겸손해 할 줄 아는 성향\n정서안정성 미래에 대한 긍정적 인식과 자신에 대한 믿음을 바탕으로 타인과 관계를 구축해 나가며 예상치 못한 상황에 처하더라도 심리적 안정을 유지\n할 수 있는 성

### Initialize connection to database

In [12]:
from astrapy.db import AstraDB

# Initialization
db = AstraDB(
  token=ASTRA_DB_APPLICATION_TOKEN,
  api_endpoint="https://1bbc37c9-e5be-4ee7-ad11-f7d5e9d9c085-us-east1.apps.astra.datastax.com")

print(f"Connected to Astra DB: {db.get_collections()}")

Connected to Astra DB: {'status': {'collections': []}}


In [13]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

### Create LangChain embedding and LLM objects for later usage:

In [14]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/Users/syshin/Desktop/Syshin/Study/MLDL/LangChain/crash_course/pdf_query/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/syshin/Desktop/Syshin/Study/MLDL/LangChain/crash_course/pdf_query/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `fr

### Create LangChain vencor store backed by Astra DB

In [15]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

In [16]:
from langchain.text_splitter import CharacterTextSplitter

# split the text using Character Text Split such that i should not increase token size
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)

texts = text_splitter.split_text(raw_text)
texts[:50]

['2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 1/5\n사람인 인성검사 결과응시자\n신승엽희망직무\nIT·인터넷검사일자\n2024.01.21\n표준점수 인성검사 응시자 집단에서 나의 상대적인 위치를 나타내기 위해 산출하는 점수\n백분위 나보다 낮은 점수를 받은 사람을 백분율로 표기\n예) 점수가 95%라면 내 점수보다 낮은 사람이 95% 있다는 것을 의미\n응답신뢰도 일관적인 답변을 하지 않았거나 실제 자신의 성향보다 더 긍정적인 방향으로 응답하면 신뢰도가 낮아질 수 있음직무적합(IT·인터넷) : 높음\nIT·인터넷직에서는 작은 것도 놓치지 않는 세밀함, 논리적이고 체계적인 사고력 및 업무적으로 타인에게 신뢰감을 심어줄 수\n있는 자세, 주도적인 태도로 새롭거나 대안적인 방안을 제시하려는 자세가 요구됩니다.\n신승엽님은 해당 직무에 대한 적합도가 매우 높아 주어진 직무 상황에 잘 적응하며, 우수한 역량을 발휘할 가능성이 높습니다.* 본 검사는 응답을 토대로 작성되었으며, 본인의 직무 적합도를 확인할 수 있습니다. 종합 결과\n표준점수\n69 점백분위\n93 %응답신뢰도\n낮음\n인성 요인 비교\n정직겸손성\n정서안정성\n외향성\n원만성성실성개방성내 점수 평균 점수\n요인별 평가2/6/24, 7:18 PM 사람인  인성검사  - 사람인',
 '표준점수\n69 점백분위\n93 %응답신뢰도\n낮음\n인성 요인 비교\n정직겸손성\n정서안정성\n외향성\n원만성성실성개방성내 점수 평균 점수\n요인별 평가2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 2/5정직겸손성 자신에게 유리한 방향으로 의사 결정하거나 사람들을 조정하지 않고, 물질적인

### Load the dataset into the vector store

In [17]:
astra_vector_store.add_texts(texts=texts)

print("Inserted %i headlines." % len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 9 headlines.


### Run the QA cycle

Simply run the cells and ask a question -or quit to stop(you can also stop execution with the "*" button on the top toolbar)

In [20]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else: 
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()
    
    if query_text.lower() == 'quit':
        break

    if query_text == "":
        continue

    first_question = False


    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMNETS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print(f"    {score}, {doc.page_content[:84]}...")


SyntaxError: EOL while scanning string literal (4162692443.py, line 23)